# <center>[⚔️ Блендинг - смешай и скор вырастет](https://stepik.org/lesson/825510/)</center>

### Оглавление ноутбука

<img src='https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/images/blending.jpg'/>
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">😺🚀 Обучаем CatBoost </a></li>
<li><a href="#c1">🦄🎳 Обучаем LightGBM </a></li>
<li><a href="#c1">👽🔱 Обучаем XGBoost </a></li>
<li><a href="#c1">🐲 Блендинг и принципы блендинга </a></li>
<li><a href="#6">🧸 Выводы и заключения</a>

</li></ul></font></p>

    

<div class="alert alert-info">

* Основная идея данной техники заключается в том, чтобы взять от каждого алгоритма лучшее и совместить несколько разных ML моделей в одну.
* За счет такого объединения увеличивается обобщающая способность финальной модели и качество улучшается.
* Помимо этого ваша модель становится более стабильной, что позволяет не слететь на приватном лидерборде.
* Особенно хорошо накидывает блендинг, если смешиваемые **модели имеют разную природу**: пример, нейронные сети, KNN и решающие деревья, в этом случае они выучивают разные зависимости и хорошо дополняют друг друга.

## Импортируем библиотеки

In [ ]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
# Модели для смешивания
import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

## Считываем данные

In [ ]:
data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"
data = pd.read_csv(data_root + "quickstart_train.csv")

### Заменим категориальные признаки на числовые значения
cat_cols = ["model", "car_type", "fuel_type"]
for col in cat_cols:
    data[col] = data[col].replace(np.unique(data[col]), np.arange(data[col].nunique()))
    data[col] = data[col].astype("category")

data.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,8,1,1,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,23,1,1,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,16,3,1,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173
3,u29695600e,12,0,1,4.04,2011,1263,2020,32.22,engine_fuel,3.880920,1.651883e+07,0.1,172.793237,-5.029476,174,170
4,N-8915870N,16,3,1,4.70,2012,26428,2017,27.51,engine_fuel,4.181149,1.398317e+07,0.1,203.462289,-14.260456,174,171


### Разделим выборку на валидационную и обучающую

In [ ]:
cols2drop = ["car_id", "target_reg", "target_class"]

X_train, X_val, y_train, y_val = train_test_split(
    data.drop(cols2drop, axis=1),
    data["target_reg"],
    test_size=0.25,
    stratify=data["target_class"],
    random_state=42,
)
print(X_train.shape, X_val.shape)

(1752, 14) (585, 14)


# <center> Обучим три модели для смешивания - блендинга

## 😺🚀 Обучаем **`CatBoost`**

In [ ]:
params_cat = {
    "n_estimators": 1500,
    "learning_rate": 0.03,
    "depth": 3,
    "use_best_model": True,
    "cat_features": cat_cols,
    "text_features": [],
    # 'train_dir' : '/path/to/catboost/model',
    "border_count": 64,
    "l2_leaf_reg": 1,
    "bagging_temperature": 2,
    "rsm": 0.5,
    "loss_function": "RMSE",  # Не определена для регресси
    # 'auto_class_weights' : 'Balanced', # Не опредлен для регресси
    "random_state": 42,
    "custom_metric": ["MAE", "MAPE"],
}

cat_model = cb.CatBoostRegressor(**params_cat)

In [ ]:
cat_model.fit(
    X_train,
    y_train,
    verbose=100,
    eval_set=(X_val, y_val),
    early_stopping_rounds=150,
)

0:	learn: 17.4391776	test: 17.9234161	best: 17.9234161 (0)	total: 47.9ms	remaining: 1m 11s
100:	learn: 12.0171853	test: 12.3281023	best: 12.3281023 (100)	total: 127ms	remaining: 1.76s
200:	learn: 11.4189213	test: 11.7777692	best: 11.7777692 (200)	total: 211ms	remaining: 1.36s
300:	learn: 11.1134309	test: 11.6124163	best: 11.6124163 (300)	total: 407ms	remaining: 1.62s
400:	learn: 10.8590320	test: 11.5378271	best: 11.5365214 (398)	total: 519ms	remaining: 1.42s
500:	learn: 10.6685339	test: 11.5151383	best: 11.5129698 (494)	total: 655ms	remaining: 1.31s
600:	learn: 10.5119473	test: 11.5108646	best: 11.4979901 (561)	total: 740ms	remaining: 1.11s
700:	learn: 10.3486559	test: 11.5041313	best: 11.4913259 (636)	total: 817ms	remaining: 932ms
Stopped by overfitting detector  (150 iterations wait)

bestTest = 11.49132595
bestIteration = 636

Shrink model to first 637 iterations.


In [ ]:
print('MSE (catboost) : ', mean_squared_error(cat_model.predict(X_val), y_val).round(3))

MSE (catboost) :  132.051


In [ ]:
# Сравним с бейзлайном в виде среднего значения
mean_squared_error(np.ones(len(y_val)) * y_val.mean(), y_val).round(3)

323.939

In [ ]:
submit = pd.DataFrame({"target": cat_model.predict(X_val).reshape(-1)})
#submit.to_csv("../tmp_data/catboost_preds.csv", index=False)
submit.to_csv("catboost_preds.csv", index=False)
submit.head()

,target
0,32.987894
1,47.608527
2,35.089056
3,61.962611
4,71.456127


## 🦄🎳 Обучаем `LightGBM`

In [ ]:
params_lgbm = {
    "num_leaves": 200,
    "n_estimators": 1500,
    # "max_depth": 7,
    "min_child_samples": 2073,
    "learning_rate": 0.0051,
    "min_data_in_leaf": 10,
    "feature_fraction": 0.99,
    "categorical_feature": cat_cols,
    'reg_alpha' : 5.0,
    'reg_lambda' : 5.0,
}

lgbm_model = lgbm.LGBMRegressor(**params_lgbm)

In [ ]:
lgbm_model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=100,
    verbose=150,
)

[LightGBM] [Warning] feature_fraction is set=0.99, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.99
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=2073 will be ignored. Current value: min_data_in_leaf=10
[150]	valid_0's l2: 189.37
[300]	valid_0's l2: 157.653
[450]	valid_0's l2: 144.558
[600]	valid_0's l2: 139.638
[750]	valid_0's l2: 139.828


LGBMRegressor(categorical_feature=['model', 'car_type', 'fuel_type'],
              feature_fraction=0.99, learning_rate=0.0051,
              min_child_samples=2073, min_data_in_leaf=10, n_estimators=1500,
              num_leaves=200, reg_alpha=5.0, reg_lambda=5.0)

In [ ]:
print('MSE (lgb) : ', mean_squared_error(lgbm_model.predict(X_val), y_val).round(3))

MSE (lgb) :  139.132


In [ ]:
submit = pd.DataFrame({"target": lgbm_model.predict(X_val).reshape(-1)})
#submit.to_csv("../tmp_data/lgbm_preds.csv", index=False)
submit.to_csv("lgbm_preds.csv", index=False)
submit.head()

,target
0,33.660865
1,42.797476
2,34.142549
3,64.390769
4,66.186459


## 👽🔱 Обучаем `XGBoost`

In [ ]:
# xgboost не умеет работать с категориальными признаками, так что нужно сделать ohe
X_train = pd.get_dummies(X_train, columns=["car_type", "fuel_type", "model"])
X_val = pd.get_dummies(X_val, columns=["car_type", "fuel_type", "model"])
X_train.shape

(1752, 43)

In [ ]:
params_xgb = {
    "eta": 0.05,
    "max_depth": 5,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "objective": "reg:linear",
    "eval_metric": "mae",
    'tree_method' : 'hist', # Supported tree methods for cat fs are `gpu_hist`, `approx`, and `hist`.
    'enable_categorical' : True

}

xgb_model = xgb.XGBRegressor(**params_xgb)

In [ ]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    early_stopping_rounds=100,
    verbose=25,
)
print('best_iteration', xgb_model.best_iteration)

[22:10:01] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:42.24252	validation_1-mae:42.20440
[25]	validation_0-mae:12.70880	validation_1-mae:12.98292
[50]	validation_0-mae:7.79993	validation_1-mae:9.14011
[75]	validation_0-mae:6.97940	validation_1-mae:8.76001
[99]	validation_0-mae:6.55026	validation_1-mae:8.74984
best_iteration 90


In [ ]:
print('MSE (xgboost) : ', mean_squared_error(xgb_model.predict(X_val), y_val).round(3))

MSE (xgboost) :  137.036


In [ ]:
submit = pd.DataFrame({"target": xgb_model.predict(X_val).reshape(-1)})
#submit.to_csv("../tmp_data/xgb_preds.csv", index=False)
submit.to_csv("xgb_preds.csv", index=False)
submit.head()

,target
0,32.807278
1,43.922146
2,33.644127
3,60.386654
4,69.922684


# <center> 😈 А теперь блендим модели!

* __Быстрый и простой способ сбледнить - это усреднить ответы!__ <br>
* __Но мы сразу покажем смешивание с весами моделей - взвешивание__

In [ ]:
#cb_model  = pd.read_csv("../tmp_data/catboost_preds.csv")["target"]  # 132.051 Лучшая (ставим ей больший вес)
#xgb_model = pd.read_csv("../tmp_data/xgb_preds.csv")["target"]       # 137.844 Средняя
#lgb_model = pd.read_csv("../tmp_data/lgbm_preds.csv")["target"]      # 139.132 Худшая (ставим ей меньший вес)
cb_model  = pd.read_csv("catboost_preds.csv")["target"]  # 132.051 Лучшая (ставим ей больший вес)
xgb_model = pd.read_csv("xgb_preds.csv")["target"]       # 137.844 Средняя
lgb_model = pd.read_csv("lgbm_preds.csv")["target"]      # 139.132 Худшая (ставим ей меньший вес)

In [ ]:
# score1 > score2 > score3 : w1 > w2 > w3
ensemble = cb_model * 0.50 + xgb_model * 0.35 + lgb_model * 0.15

In [ ]:
print('MSE (ensemble) :', mean_squared_error(ensemble, y_val).round(3))

MSE (ensemble) : 131.85


In [ ]:
round((100*(132.051 - 131.080)/132.051), 1)

0.7

# <center> 🥳  Ура, скор улучшился!
    
<div class="alert alert-info">
    
**Как подбирать веса?**
- Опираясь на скоры на лидер-борде
- По локальной валидации
- Веса пропорциональны скору

### Если хочется кодом, то вот более универсальный способ

In [ ]:
weights = {"catboost": 0.5, "lgbm": 0.15, "xgb": 0.35}

In [ ]:
import os

preds = pd.DataFrame()

# соберем единый датафрейм из наших предсказаний
for model_name in ["catboost", "lgbm", "xgb"]:

    #path = os.path.join("../tmp_data/", f"{model_name}_preds.csv")
    path = os.path.join("", f"{model_name}_preds.csv")
    now = pd.read_csv(path).reset_index()

    now["model"] = model_name
    now["target"] *= weights[model_name]
    preds = pd.concat([preds, now])

preds.head()

,index,target,model
0,0,16.493947,catboost
1,1,23.804264,catboost
2,2,17.544528,catboost
3,3,30.981306,catboost
4,4,35.728063,catboost


In [ ]:
preds["model"].unique()

array(['catboost', 'lgbm', 'xgb'], dtype=object)

In [ ]:
ensemble = preds.groupby("index")["target"].agg("sum")
mean_squared_error(ensemble, y_val)

131.84991442662573

# 🐲 Принципы блендинга (просто и логично)

<div class="alert alert-info">

* 🦑 Не бленди пока не выжал максимум из моделей по отдельности
* 🐳 Чем различнее и сильнее модели, тем эффективнее блендинг (⚠️ эффективность)
* 🐙 При равной точности, ансабль побеждает соло-модель на привате (⚠️ стабильность)
* 🦐 Чем раньше проверишь эффект от блендинга, тем эффективне будет стратегия
* 🐠 Блендить можно с разными весами пропорционально скорам или разным фичам
* 🐋 Блендинг по фолдам и чекпоинтам обучения - это тоже блендинг
* 🐡 Блендинг по сидам - это стабилизирующий блендинг
* 🐬 Против блендинга только больший блендинг

<div class="alert alert-info">
    
__забегая вперёд__
    
* 🦈 Стекинг сильнее блендинга, но капризнее
* 🎣 Стекать можно с фичами
* 🎏 Стекинг бывает разных уровней
* 🐉 Не можешь больше стакать - блендись с решением сокомандников

# 🧸 Выводы и заключения

<div class="alert alert-info">

* Блендинг это сильный инструмент, который зачастую неплохо поднимаем качество моделей.
* При этом само смешивание провести можно вообще в одну строчку, просто взяв среднее моделей и взвесив
* `ensemble = model1 * w1 + model2 * w2 + model3 * w3 + ...`